# Baran Experiment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import IPython.display

import raha
import datetime
from ruska import Ruska

In [8]:
rsk = Ruska(name='2022-06-01-nursery-jenga-diligent',
             description='''Die Messung auf dem breast-cancer-Datensatz hat nicht funktioniert, Jenga
             korrumpiert die Daten nicht richtig. Ich versuche es jetzt mit dem etwas größeren Nursery
             Datensatz.''',
             commit='520e7f83508f4bbab186b95134c07285d1b8bf01',
             config={'dataset': 'nursery',
                     'sampling': 'MCAR',
                     'error_fraction': .03,
                     'labeling_budget': 20,
                     'feature_generators': ['value', 'domain', 'vicinity'],
                     'classification_model': 'ABC',
                     'vicinity_orders': [1, 2],
                     'vicinity_feature_generator': 'pdep',
                     'n_best_pdeps': 5,
                     'score_strategy': 'multiply',
                     },
             ranges={'sampling': ['MCAR', 'MAR', 'MNAR'],
                     'error_fraction': [.03, .05, .1, .2, .3, .4, .5],
                     },
             runs=3,
             save_path='/Users/philipp/code/experimente/2022W21-Jenga-Structured-Approach/'
            )

In [9]:
def run_baran(dataset, sampling, error_fraction, labeling_budget, feature_generators, classification_model, vicinity_orders, vicinity_feature_generator, n_best_pdeps, score_strategy, run):
    rate_formatted = str(error_fraction).split('.')[1]
    data_dict = {"name": dataset,
                 "path": f"../datasets/{dataset}/{sampling}/dirty_{rate_formatted}.csv",
                 "clean_path": f"../datasets/{dataset}/clean.csv"}

    data = raha.Dataset(data_dict)
    data.detected_cells = dict(data.get_actual_errors_dictionary())
    app = raha.Correction()
    app.LABELING_BUDGET = labeling_budget
    app.VERBOSE = False
    app.FEATURE_GENERATORS = feature_generators
    app.CLASSIFICATION_MODEL = classification_model
    app.VICINITY_ORDERS = vicinity_orders
    app.VICINITY_FEATURE_GENERATOR = vicinity_feature_generator
    app.N_BEST_PDEPS = n_best_pdeps
    app.PDEP_SCORE_STRATEGY = score_strategy

    d = app.initialize_dataset(data)
    app.initialize_models(d)
    while len(d.labeled_tuples) < app.LABELING_BUDGET:
        app.sample_tuple(d, random_seed=None)
        app.label_with_ground_truth(d)
        app.update_models(d)
        app.generate_features(d, synchronous=True)
        app.predict_corrections(d, random_seed=None)

        p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
    return {'precision': p, 'recall': r, 'f1': f}

In [10]:
rsk.run(experiment=run_baran)

Run 1/63. 50s per run, estimate 51m 27s to finish
Run 2/63. 52s per run, estimate 52m 38s to finish
Run 3/63. 52s per run, estimate 51m 36s to finish
Run 4/63. 52s per run, estimate 50m 18s to finish
Run 5/63. 51s per run, estimate 49m 18s to finish
Run 6/63. 51s per run, estimate 48m 12s to finish
Run 7/63. 52s per run, estimate 48m 9s to finish
Run 8/63. 53s per run, estimate 47m 47s to finish
Run 9/63. 53s per run, estimate 47m 20s to finish
Run 10/63. 54s per run, estimate 47m 14s to finish
Run 11/63. 3m 50s per run, estimate 3h 15m 48s to finish
Run 12/63. 3m 36s per run, estimate 3h 29s to finish
Run 13/63. 3m 25s per run, estimate 2h 48m 9s to finish
Run 14/63. 3m 17s per run, estimate 2h 38m 7s to finish
Run 15/63. 3m 9s per run, estimate 2h 28m 45s to finish
Run 16/63. 3m 4s per run, estimate 2h 21m 32s to finish
Run 17/63. 2m 59s per run, estimate 2h 14m 25s to finish
Run 18/63. 2m 54s per run, estimate 2h 8m 4s to finish
Run 19/63. 2m 50s per run, estimate 2h 2m 27s to finis

# ruska naive

In [11]:
rsk_naive = Ruska(name='2022-06-01-nursery-jenga-diligent-naive',
             description='''Die Messung auf dem breast-cancer-Datensatz hat nicht funktioniert, Jenga
             korrumpiert die Daten nicht richtig. Ich versuche es jetzt mit dem etwas größeren Nursery
             Datensatz.''',
             commit='ca6f5e8a4904fdaa2f7c862590cda9a7b09dfb84',
             config={'dataset': 'nursery',
                     'sampling': 'MCAR',
                     'error_fraction': .03,
                     'labeling_budget': 20,
                     'feature_generators': ['value', 'domain', 'vicinity'],
                     'classification_model': 'ABC',
                     'vicinity_orders': [1],
                     'vicinity_feature_generator': 'naive',
                     'n_best_pdeps': 5,
                     'score_strategy': 'multiply',
                     },
             ranges={'sampling': ['MCAR', 'MAR', 'MNAR'],
                     'error_fraction': [.03, .05, .1, .2, .3, .4, .5],
                     },
             runs=3,
             save_path='/Users/philipp/code/experimente/2022W21-Jenga-Structured-Approach/'
            )

In [12]:
rsk_naive.run(experiment=run_baran)

Run 1/63. 3s per run, estimate 3m 54s to finish
Run 2/63. 3s per run, estimate 3m 45s to finish
Run 3/63. 3s per run, estimate 3m 41s to finish
Run 4/63. 3s per run, estimate 3m 40s to finish
Run 5/63. 3s per run, estimate 3m 39s to finish
Run 6/63. 3s per run, estimate 3m 36s to finish
Run 7/63. 4s per run, estimate 4m 33s to finish
Run 8/63. 5s per run, estimate 5m 20s to finish
Run 9/63. 6s per run, estimate 5m 43s to finish
Run 10/63. 7s per run, estimate 6m 29s to finish
Run 11/63. 8s per run, estimate 6m 53s to finish
Run 12/63. 8s per run, estimate 7m 14s to finish
Run 13/63. 10s per run, estimate 8m 33s to finish
Run 14/63. 12s per run, estimate 9m 36s to finish
Run 15/63. 13s per run, estimate 10m 22s to finish
Run 16/63. 14s per run, estimate 11m 14s to finish
Run 17/63. 15s per run, estimate 11m 56s to finish
Run 18/63. 17s per run, estimate 12m 33s to finish
Run 19/63. 18s per run, estimate 13m 27s to finish
Run 20/63. 20s per run, estimate 14m 9s to finish
Run 21/63. 21s p